In [160]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

## NSC Data

In [3]:
filters = ['g','r','i','z'];
df_dict = {};
objs_dict = {};
N_lc_dict = {};

for fil in filters:
    df_dict[fil] = pd.read_csv('../data/lightCurves/lc_'+fil+'_n100.csv');
    objs_dict[fil] = df_dict[fil].objectid.unique();
    N_lc_dict[fil] = objs_dict[fil].shape[0];
    
    print("filter:", fil)
    print("# of objects:", N_lc_dict[fil])

filter: g
# of objects: 3658
filter: r
# of objects: 6157
filter: i
# of objects: 4714
filter: z
# of objects: 3976


In [4]:
%%time
X_raw = {};

for fil in filters:
    selcol = df_dict[fil][['mjd', 'mag_aper2', 'magerr_aper2']].to_numpy() # Select columns and convert to np.array
    X_raw[fil] = np.array([selcol[df_dict[fil].objectid.to_numpy() == objs_dict[fil][i]][:100] for i in range(len(objs_dict[fil]))])

Wall time: 6min 32s


In [5]:
def preprocess(arr):
    """
    Preprocess light curve data
    
    Arguments:
        arr: np.array of data
    
    Returns:
        X: output data
    """
    
    X = arr.copy()
    mins = np.atleast_2d(np.amin(X[:,:,0], axis=1)).T     # Find smallest mjd
    X[:,:,0] -= mins                                      # Shift mjd to start at 0
    
    return X

In [6]:
X = {};
for fil in filters:
    X[fil] = preprocess(X_raw[fil])

In [7]:
for fil in filters:
    print(fil, X[fil].shape)

g (3658, 100, 3)
r (6157, 100, 3)
i (4714, 100, 3)
z (3976, 100, 3)


In [97]:
X_all = np.concatenate([X[fil] for fil in filters])
X_all.shape

(18505, 100, 3)

In [10]:
for fil in filters:
    np.save('../data/lightCurves/lc_'+fil+'_n100', X[fil])

In [98]:
np.save('../data/lightCurves/lc_griz_n100', X_all)

In [105]:
bar = np.load('../data/lightCurves/lc_griz_n100.npy')

In [106]:
bar.shape

(18505, 100, 3)